In [1]:
from docling.document_converter import DocumentConverter

source = "C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx"  # can be .pdf, .docx, .xlsx
converter = DocumentConverter()
doc = converter.convert(source).document

# Export to different formats depending on your RAG needs
text_md = doc.export_to_markdown()


#print(text_md)


c:\Users\sarojd\AppData\Local\anaconda3\envs\azure\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-11 15:22:52,755 - INFO - detected formats: [<InputFormat.XLSX: 'xlsx'>]
c:\Users\sarojd\AppData\Local\anaconda3\envs\azure\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
2025-09-11 15:22:52,945 - INFO - Going to convert document batch...
2025-09-11 15:22:52,946 - INFO - Initializing pipeline for SimplePipeline with options hash 64efdeb459220d9bbf631bfc20c2d4cd
2025-09-11 15:22:52,946 - INFO - Processing document YieldAction.xlsx
2025-09-11 15:22:52,947 - INFO - Processing sheet: Cover
2025-09-11 15:22:52,964 - INFO - Processing sheet: Datalink
2025-09-11 15:22:52,965 - INFO - Processing sheet: Yield Summary
2025-09-1

In [ ]:
import pandas as pd
import os
from docling.document_converter import DocumentConverter

source = "C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx"
temp_file = "temp_truncated.xlsx"

# Load and truncate each sheet to 10,000 rows
with pd.ExcelWriter(temp_file, engine="openpyxl") as writer:
    xls = pd.ExcelFile(source)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(source, sheet_name=sheet_name)
        df.head(10000).to_excel(writer, sheet_name=sheet_name, index=False)

# Now run Docling on the truncated version
converter = DocumentConverter()
doc = converter.convert(temp_file).document

text_md = doc.export_to_markdown()
#print(text_md)

# Clean up if needed
os.remove(temp_file)


c:\Users\sarojd\AppData\Local\anaconda3\envs\azure\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
2025-09-10 09:58:30,140 - INFO - detected formats: [<InputFormat.XLSX: 'xlsx'>]
2025-09-10 09:58:33,505 - INFO - Going to convert document batch...
2025-09-10 09:58:33,506 - INFO - Initializing pipeline for SimplePipeline with options hash 64efdeb459220d9bbf631bfc20c2d4cd
2025-09-10 09:58:33,506 - INFO - Processing document temp_truncated.xlsx
2025-09-10 09:58:33,508 - INFO - Processing sheet: Cover
2025-09-10 09:58:33,509 - INFO - Processing sheet: Datalink
2025-09-10 09:58:33,510 - INFO - Processing sheet: Yield Summary
2025-09-10 09:58:33,513 - INFO - Processing sheet: Yield Analysis
2025-09-10 09:58:33,514 - INFO - Processing sheet: Yield Actions Summary
2025-09-10 09:58:33,601 - INFO - Processing sheet: SN
2025-09-10 09:58:33,603 - INFO - Processing sheet: References
2025-09-10 09:58:33,605 -

In [2]:
#print(text_md)
from langchain.schema import Document

# Create a single LangChain Document
lc_doc = Document(page_content=text_md, metadata={"source": source})
print(lc_doc)


page_content='| Yield  Summary (by phase and product)   | Yield  Summary (by phase and product)   | Yield  Summary (by phase and product)   |
|-----------------------------------------|-----------------------------------------|-----------------------------------------|
| ADVA Number                             | CM PN                                   | Station                                 |
| ADV-4099308001                          | ADV-4099309001                          | EM 1.1 LOT 1877                         |
| SMOD F8 RD-1RS MB V1.0+uCM-RT+CC3 V1.4A | AS PCB F8 RD-1RS MB V1.0                | 1st Pass yield                          |
| SMOD F8 RD-1RS MB V1.0+uCM-RT+CC3 V1.4A | AS PCB F8 RD-1RS MB V1.0                | Tested                                  |
| SMOD F8 RD-1RS MB V1.0+uCM-RT+CC3 V1.4A | AS PCB F8 RD-1RS MB V1.0                | Failed                                  |

| SMT1 (B)                     | SMT2 (T)                     | 5Dx                      

In [ ]:
from docling.document_converter import DocumentConverter

doc = DocumentConverter().convert("C:/Users/sarojd/AZURE/deployDocPortalDemo/data/Build report 1912.docx").document

# Each image is stored in resources
for i, img in enumerate(doc.resources.images):
    with open(f"image_{i}.png", "wb") as f:
        f.write(img.data)


In [1]:
from langchain_excel_loader import StructuredExcelLoader

loader = StructuredExcelLoader("C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx")
docs=loader.load()
print(len(docs))

c:\Users\sarojd\AppData\Local\anaconda3\envs\azure\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


9


In [ ]:
# Preview the first few documents  
for i, doc in enumerate(docs):  # adjust the slice if you want more  
    print(f"--- Document {i+1} ---")  
    print("Metadata:", doc.metadata)  
    print("Content:", doc.page_content)  
    print() 

In [4]:
from openpyxl import load_workbook  
from langchain.schema import Document  
  
source = "C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx"  
wb = load_workbook(source, data_only=True)  
  
docs = []  
  
for sheet in wb.sheetnames:  
    ws = wb[sheet]  
  
    # Get header row (first row of the sheet)  
    try:  
        headers = [  
            str(cell).strip() if cell is not None else ""  
            for cell in next(ws.iter_rows(values_only=True))  
        ]  
    except StopIteration:  
        # Empty sheet, skip  
        continue  
  
    # Iterate remaining rows  
    for row in ws.iter_rows(values_only=True):  
        row_data = {  
            headers[i]: (str(cell).strip() if cell is not None else "")  
            for i, cell in enumerate(row)  
        }  
  
        # Skip completely empty rows  
        if not any(row_data.values()):  
            continue  
  
        # Create text representation for embeddings  
        text_repr = " | ".join(f"{k}: {v}" for k, v in row_data.items())  
  
        docs.append(  
            Document(  
                page_content=text_repr,  
                metadata={  
                    "source": source,  
                    "sheet": sheet,  
                    **row_data  # include each column as metadata  
                }  
            )  
        )  
  
print(f"Created {len(docs)} row-level documents")  
print(docs[0])

Created 86 row-level documents
page_content=': R&D found MFG failure' metadata={'source': 'C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx', 'sheet': 'Datalink', '': 'R&D found MFG failure'}


In [5]:
print(docs)

[Document(metadata={'source': 'C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx', 'sheet': 'Datalink', '': 'R&D found MFG failure'}, page_content=': R&D found MFG failure'), Document(metadata={'source': 'C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx', 'sheet': 'Datalink', '': '#REF!'}, page_content=': #REF!'), Document(metadata={'source': 'C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx', 'sheet': 'Yield Summary', 'Yield  Summary (by phase and product)': 'Yield  Summary (by phase and product)', '': ''}, page_content='Yield  Summary (by phase and product): Yield  Summary (by phase and product) | : '), Document(metadata={'source': 'C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx', 'sheet': 'Yield Summary', 'Yield  Summary (by phase and product)': 'ADVA Number', '': 'Internal Defects'}, page_content='Yield  Summary (by phase and product): ADVA Number | : Internal Defects'), Document(metadata={'source': 'C:/Users/sarojd/AZURE

In [1]:
import pandas as pd  
from langchain.schema import Document  
from pathlib import Path

def load_xlsx_as_markdown(path: Path) -> list[Document]:  
    docs = []  
    xls = pd.ExcelFile(path)  
    for sheet_name in xls.sheet_names:  
        df = pd.read_excel(path, sheet_name=sheet_name, dtype=str)  # keep as strings  
        df = df.fillna("")  # avoid NaNs  
        markdown_table = df.to_markdown(index=False)  # Markdown format for LLM  
        docs.append(  
            Document(  
                page_content=markdown_table,  
                metadata={  
                    "source": str(path),  
                    "sheet": sheet_name,  
                    "type": "table"  
                }  
            )  
        )  
    return docs  
print(load_xlsx_as_markdown("C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx"))


c:\Users\sarojd\AppData\Local\anaconda3\envs\azure\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


[Document(metadata={'source': 'C:/Users/sarojd/AZURE/deployDocPortalDemo/data/YieldAction.xlsx', 'sheet': 'Cover', 'type': 'table'}, page_content='| Unnamed: 0   | Unnamed: 1                                                                                                         | Unnamed: 2                               | Unnamed: 3            | Unnamed: 4                               | Unnamed: 5   | Unnamed: 6   | Unnamed: 7   | Unnamed: 8          |\n|:-------------|:-------------------------------------------------------------------------------------------------------------------|:-----------------------------------------|:----------------------|:-----------------------------------------|:-------------|:-------------|:-------------|:--------------------|\n|              | Document Number                                                                                                    |                                          |                       | Document Title              